## Домашняя работа №3

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.models import Sequential
import tensorflow.keras.metrics

In [ ]:
print('TensorFlow version is ' + tf.__version__)

TensorFlow version is 2.5.0


In [2]:
boston_data = load_boston()
df_boston = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
df_boston['target'] = pd.Series(boston_data.target)
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
X, y = load_boston(return_X_y=True)
print(y.shape)

(506,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [5]:
X = tf.constant( X_train , dtype=tf.float32 )
Y = tf.constant( y_train , dtype=tf.float32 ) 

In [6]:
num_features = X_train.shape[1]
num_classes = len(np.unique(y_train))

In [7]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(X))

In [ ]:
first = np.array(X_train[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[  0.03   0.     2.18   0.     0.46   6.43  58.7    6.06   3.   222.
   18.7  394.12   5.21]]

Normalized: [[-0.41 -0.51 -1.29 -0.28 -0.85  0.15 -0.37  1.08 -0.75 -1.11  0.19  0.4
  -1.02]]


Построение первой модели

In [10]:
model = Sequential([
  normalizer,
  Dense(128, activation='tanh', input_shape=(num_features,)),
  Dense(128, activation='tanh'),
  Dense(1),
])


model.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=['mae'],
)

model_fit = model.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [11]:
def history_last(model):
    last = list(model.history.history.values())[-1][-1]
    return last

In [12]:
first_note = [{"name": 'model', "loss": 'mse', 'metrics': 'mae', 'metric_value': history_last(model)}]

In [13]:
model_df = pd.DataFrame(first_note)

In [ ]:
model_df

,name,loss,metrics,metric_value
0,model,mse,mae,2.251323
1,model1,logcosh,mae,2.578054
2,model2,mse,mae,2.217207
3,model3,msle,mae,4.075542
4,model4,msle,mae,3.881510
5,model5,mse,mape,11.108685
6,model6,mse,rmse,3.328029
7,model7,mae,mae,2.534641


In [14]:
model1 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='tanh'),
  Dense(1),
])


model1.compile(
  optimizer='SGD',
  loss='logcosh',
  metrics=['mae'],
)

model_fit = model1.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [15]:
model_df.loc[len(model_df)] = ['model1', 'logcosh', 'mae', history_last(model1)]

In [16]:
model2 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model2.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=['mae'],
)

model_fit = model2.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [17]:
model_df.loc[len(model_df)] = ['model2', 'mse', 'mae', history_last(model2)]

In [18]:
model3 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model3.compile(
  optimizer='SGD',
  loss='mean_squared_logarithmic_error',
  metrics=['mae'],
)

model_fit = model3.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [19]:
model_df.loc[len(model_df)] = ['model3', 'msle', 'mae', history_last(model3)]

In [35]:
model4 = Sequential([
  normalizer,
  Dense(128, activation='linear', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model4.compile(
  optimizer='SGD',
  loss='mae',
  metrics=['mae'],
)

model_fit = model4.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 1s 11ms/step - loss: 22.1919 - mae: 22.1919 - val_loss: 20.9397 - val_mae: 20.9397
Epoch 2/50
19/19 [==============================] - 0s 3ms/step - loss: 19.7619 - mae: 19.7619 - val_loss: 17.7259 - val_mae: 17.7259
Epoch 3/50
19/19 [==============================] - 0s 4ms/step - loss: 14.8673 - mae: 14.8673 - val_loss: 10.6225 - val_mae: 10.6225
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 9.7912 - mae: 9.7912 - val_loss: 8.1081 - val_mae: 8.1081
Epoch 5/50
19/19 [==============================] - 0s 3ms/step - loss: 7.5315 - mae: 7.5315 - val_loss: 6.2381 - val_mae: 6.2381
Epoch 6/50
19/19 [==============================] - 0s 4ms/step - loss: 5.3246 - mae: 5.3246 - val_loss: 5.5042 - val_mae: 5.5042
Epoch 7/50
19/19 [==============================] - 0s 4ms/step - loss: 4.5381 - mae: 4.5381 - val_loss: 4.6669 - val_mae: 4.6669
Epoch 8/50
19/19 [==============================] - 0s 3ms/step - loss: 4.123

In [36]:
model_df.loc[len(model_df)] = ['model4', 'mae', 'mae', history_last(model4)]

In [37]:
model5 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model5.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=[tf.keras.metrics.MeanAbsolutePercentageError()],
)

model_fit = model5.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 1s 11ms/step - loss: 464.2149 - mean_absolute_percentage_error: 84.4837 - val_loss: 379.9748 - val_mean_absolute_percentage_error: 75.1527
Epoch 2/50
19/19 [==============================] - 0s 5ms/step - loss: 324.3831 - mean_absolute_percentage_error: 63.7824 - val_loss: 303.2720 - val_mean_absolute_percentage_error: 62.6729
Epoch 3/50
19/19 [==============================] - 0s 4ms/step - loss: 192.9001 - mean_absolute_percentage_error: 41.6667 - val_loss: 21.2807 - val_mean_absolute_percentage_error: 12.7321
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 29.6777 - mean_absolute_percentage_error: 17.9928 - val_loss: 24.7785 - val_mean_absolute_percentage_error: 16.3182
Epoch 5/50
19/19 [==============================] - 0s 4ms/step - loss: 87.5693 - mean_absolute_percentage_error: 26.1901 - val_loss: 56.8368 - val_mean_absolute_percentage_error: 20.9838
Epoch 6/50
19/19 [==============================] - 0s

In [38]:
model_df.loc[len(model_df)] = ['model5', 'mse', 'mape', history_last(model5)]

In [39]:
model6 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model6.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

model_fit = model6.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

Epoch 1/50
19/19 [==============================] - 0s 10ms/step - loss: 389.3636 - root_mean_squared_error: 19.7323 - val_loss: 364.0063 - val_root_mean_squared_error: 19.0789
Epoch 2/50
19/19 [==============================] - 0s 4ms/step - loss: 155.2312 - root_mean_squared_error: 12.4592 - val_loss: 550.4827 - val_root_mean_squared_error: 23.4624
Epoch 3/50
19/19 [==============================] - 0s 4ms/step - loss: 258.5624 - root_mean_squared_error: 16.0799 - val_loss: 271.1424 - val_root_mean_squared_error: 16.4664
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 191.6046 - root_mean_squared_error: 13.8421 - val_loss: 27.6225 - val_root_mean_squared_error: 5.2557
Epoch 5/50
19/19 [==============================] - 0s 5ms/step - loss: 62.8342 - root_mean_squared_error: 7.9268 - val_loss: 28.5764 - val_root_mean_squared_error: 5.3457
Epoch 6/50
19/19 [==============================] - 0s 4ms/step - loss: 43.1898 - root_mean_squared_error: 6.5719 - val_loss:

In [40]:
model_df.loc[len(model_df)] = ['model6', 'mse', 'rmse', history_last(model6)]

In [41]:
model7 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model7.compile(
  optimizer='SGD',
  loss='mae',
  metrics=['mae'],
)

model_fit = model7.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [42]:
model_df.loc[len(model_df)] = ['model7', 'mae', 'mae', history_last(model7)]

In [43]:
model8 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model8.compile(
  optimizer='SGD',
  loss='mae',
  metrics=[tf.keras.metrics.MeanAbsolutePercentageError()],
)

model_fit = model8.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 1s 28ms/step - loss: 22.4664 - mean_absolute_percentage_error: 96.9558 - val_loss: 21.4038 - val_mean_absolute_percentage_error: 93.3316
Epoch 2/50
19/19 [==============================] - 0s 4ms/step - loss: 20.5901 - mean_absolute_percentage_error: 87.0346 - val_loss: 18.8156 - val_mean_absolute_percentage_error: 80.2189
Epoch 3/50
19/19 [==============================] - 0s 4ms/step - loss: 16.2172 - mean_absolute_percentage_error: 65.4155 - val_loss: 11.8867 - val_mean_absolute_percentage_error: 49.2851
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 8.6477 - mean_absolute_percentage_error: 37.2079 - val_loss: 5.6433 - val_mean_absolute_percentage_error: 24.8081
Epoch 5/50
19/19 [==============================] - 0s 4ms/step - loss: 5.2538 - mean_absolute_percentage_error: 24.1818 - val_loss: 4.2899 - val_mean_absolute_percentage_error: 19.8344
Epoch 6/50
19/19 [==============================] - 0s 4ms/step

In [44]:
model_df.loc[len(model_df)] = ['model8', 'mae', 'mape', history_last(model8)]

In [50]:
model9 = Sequential([
  normalizer,
  Dense(128, activation='linear', input_shape=(num_features,)),
  Dense(128, activation='tanh'),
  Dense(1),
])


model9.compile(
  optimizer='adam',
  loss='mae',
  metrics=['mae'],
)

model_fit = model9.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 1s 10ms/step - loss: 22.8885 - mae: 22.8885 - val_loss: 22.1221 - val_mae: 22.1221
Epoch 2/50
19/19 [==============================] - 0s 3ms/step - loss: 22.0907 - mae: 22.0907 - val_loss: 21.1175 - val_mae: 21.1175
Epoch 3/50
19/19 [==============================] - 0s 3ms/step - loss: 20.6868 - mae: 20.6868 - val_loss: 18.9623 - val_mae: 18.9623
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 17.5709 - mae: 17.5709 - val_loss: 14.3364 - val_mae: 14.3364
Epoch 5/50
19/19 [==============================] - 0s 4ms/step - loss: 11.8982 - mae: 11.8982 - val_loss: 7.8732 - val_mae: 7.8732
Epoch 6/50
19/19 [==============================] - 0s 4ms/step - loss: 6.6755 - mae: 6.6755 - val_loss: 4.4603 - val_mae: 4.4603
Epoch 7/50
19/19 [==============================] - 0s 4ms/step - loss: 4.8145 - mae: 4.8145 - val_loss: 4.2796 - val_mae: 4.2796
Epoch 8/50
19/19 [==============================] - 0s 4ms/step - loss:

In [55]:
model_df.loc[9] = ['lintanh', 'mae', 'mae', history_last(model9)]

In [56]:
model_df

,name,loss,metrics,metric_value
0,model,mse,mae,2.088878
1,model1,logcosh,mae,2.624208
2,model2,mse,mae,2.551961
3,model3,msle,mae,3.993636
4,model4,mae,mae,2.609521
5,model5,mse,mape,13.088747
6,model6,mse,rmse,3.277014
7,model7,mae,mae,2.317484
8,model8,mae,mape,12.510179
9,lintanh,mae,mae,2.689232


Из документации взял команды нормализации, так же очень много посидел в разделах Loss и metrics, понял что многие метрики не подходят для задачи регрессии (напрмиер, такие как, accuracy). 

Лучше всего себя показали модели с линейными функциями активации.